In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv("train.csv") 
train_data = train_data.loc[train_data.airline_sentiment.dropna().index]
train_data = train_data.loc[train_data.text.dropna().index]

test_data = pd.read_csv("test.csv") 
test_data = test_data.loc[test_data.text.dropna().index]


In [3]:
test_data

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...
3655,570304244001193984,US Airways,NaN,Anthony_Scerri,NaN,0,@USAirways Been stuck for 40+ minutes due to l...,NaN,2015-02-24 11:28:22 -0800,"Astoria, NY",Quito
3656,567847737061941249,US Airways,NaN,mttdprkr,NaN,0,@USAirways 4 hours... 4 hours... FOUR HOURS. ...,NaN,2015-02-17 16:47:05 -0800,"Vancouver, WA",Pacific Time (US & Canada)
3657,567823564167192576,Virgin America,NaN,miaerolinea,NaN,1,Nice RT @VirginAmerica: The man of steel might...,NaN,2015-02-17 15:11:02 -0800,Worldwide,Caracas
3658,570273819287531520,American,NaN,GoldensPleasure,NaN,0,@AmericanAir Aww Thanks AA..DFW was on GMA up ...,NaN,2015-02-24 09:27:28 -0800,East Coast CT.,Central Time (US & Canada)


In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import string
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
def getStopWords():
    stop_words = stopwords.words("english")
    punctuation = list(string.punctuation)
    stop_words += punctuation
    stop_words = set(stop_words)
#     f = open("stop_words_english.txt", "r")         # opening the file in which more stop words are present 
#     a=f.read()      
#     s=a.split()               
#     stop_words.update(s) 
    return stop_words

def lemmatizer_pos_tag(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def cleaning_doc(text):
    # remove stop words1
    a=1
    output = []
    words_arr = word_tokenize(text)
    for w in words_arr:
        if w.lower() not in stop_words:
            pos = pos_tag([w])
            pos = lemmatizer_pos_tag(pos[0][1])
            final_w = lemmatizer.lemmatize(w, pos)
            output.append(final_w)
    return output

stop_words = getStopWords()

train_docs = []
train_categories = []
for i in range(train_data.shape[0]):
    text = train_data.iloc[i,7]
    train_docs.append(cleaning_doc(text))
    train_categories.append(train_data.iloc[i,1])

test_docs = []
for i in range(test_data.shape[0]):
    text = test_data.iloc[i,6]
    test_docs.append(cleaning_doc(text))

In [5]:
print(train_docs[0],train_categories[0])
print(test_docs[0])

['SouthwestAir', 'schedule', 'morning', '2', 'day', 'fact', 'yes', '..', 'sure', 'even', 'flight', 'one', 'Cancelled', 'Flightled'] negative
['AmericanAir', 'car', 'gng', 'DFW', 'Pulled', '1hr', 'ago', 'icy', 'road', 'On-hold', 'AA', 'since', '1hr', 'Ca', "n't", 'reach', 'arpt', 'AA2450', 'Wat', '2']


In [6]:
text_train_docs = [" ".join(d) for d in train_docs]
text_test_docs = [" ".join(d) for d in test_docs]

In [7]:
print(text_train_docs[0])
print(text_test_docs[0])

SouthwestAir schedule morning 2 day fact yes .. sure even flight one Cancelled Flightled
AmericanAir car gng DFW Pulled 1hr ago icy road On-hold AA since 1hr Ca n't reach arpt AA2450 Wat 2


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_features = 4000)
x_train_features = count_vectorizer.fit_transform(text_train_docs)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [9]:
count_vectorizer.get_feature_names()

['00',
 '000',
 '0016',
 '00pm',
 '02',
 '03',
 '05',
 '05pm',
 '08',
 '10',
 '100',
 '1000',
 '1007',
 '105',
 '1051',
 '1071',
 '1080',
 '1098',
 '10a',
 '10am',
 '10hrs',
 '10pm',
 '11',
 '117',
 '1171',
 '11am',
 '11pm',
 '11th',
 '12',
 '1230',
 '12am',
 '13',
 '130',
 '1357',
 '1359',
 '136',
 '1388',
 '1389',
 '13th',
 '14',
 '140',
 '1472',
 '1491',
 '15',
 '150',
 '1500',
 '1514',
 '152',
 '1533',
 '1562',
 '1583',
 '15minutes',
 '15th',
 '16',
 '1625',
 '1679',
 '17',
 '1701',
 '1706',
 '1750',
 '17th',
 '18',
 '180',
 '1800',
 '1826',
 '1861',
 '1898',
 '19',
 '1917',
 '195',
 '1970',
 '1971',
 '1am',
 '1hr',
 '1k',
 '1pm',
 '1st',
 '20',
 '200',
 '2000',
 '2012',
 '2014',
 '2015',
 '2034',
 '20min',
 '20pm',
 '21',
 '21st',
 '22',
 '2258',
 '23',
 '2324',
 '23rd',
 '24',
 '2470',
 '24h',
 '24hr',
 '24hrs',
 '24th',
 '25',
 '250',
 '26',
 '2692',
 '27',
 '275',
 '27l',
 '28',
 '28th',
 '29',
 '2955',
 '2990176298',
 '2b',
 '2d',
 '2day',
 '2days',
 '2hours',
 '2hr',
 '2hrs',

In [10]:
x_test_features = count_vectorizer.transform(text_test_docs)

In [11]:
x_test_features

<3660x4000 sparse matrix of type '<class 'numpy.int64'>'
	with 33229 stored elements in Compressed Sparse Row format>

In [12]:
# classifier 1 : 0.771
from sklearn.svm import SVC
svc = SVC(C=1.5)
svc.fit(x_train_features, train_categories)
a = svc.predict(x_test_features)
b = pd.Series(a)
b.to_csv("prediction.csv",header=False, index=False)

In [13]:
svc.score(x_train_features, train_categories)

0.9092896174863389

In [15]:
#0.7508
from sklearn.ensemble import RandomForestClassifier as SVC
svc = SVC()
svc.fit(x_train_features, train_categories)
a = svc.predict(x_test_features)
b = pd.Series(a)
b.to_csv("prediction1.csv",header=False, index=False)
svc.score(x_train_features, train_categories)

0.99408014571949

In [16]:
#0.7667
from sklearn.naive_bayes import MultinomialNB as SVC
svc = SVC(alpha=0.0001)
svc.fit(x_train_features, train_categories)
a = svc.predict(x_test_features)
b = pd.Series(a)
b.to_csv("prediction2.csv",header=False, index=False)
svc.score(x_train_features, train_categories)

0.8306010928961749

max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
max_df = 25 means "ignore terms that appear in more than 25 documents".

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vectorizer1 = TfidfVectorizer(max_features = 5000, ngram_range = (1,2), max_df=0.8, min_df=0.0001)
x_train_features = count_vectorizer1.fit_transform(text_train_docs)
x_train_features.todense()
# text_train_docs[0]
x_test_features = count_vectorizer1.transform(text_test_docs)
len(count_vectorizer1.get_feature_names())


5000

In [21]:
from sklearn.svm import SVC
svc = SVC(C=1.5)
svc.fit(x_train_features, train_categories)
a = svc.predict(x_test_features)
b = pd.Series(a)
b.to_csv("prediction3.csv",header=False, index=False)
count_vectorizer.get_feature_names()
svc.score(x_train_features, train_categories)

0.9817850637522769

In [26]:
#0.7508
from sklearn.ensemble import RandomForestClassifier as SVC
svc = SVC(n_estimators=2000, n_jobs=-1)
svc.fit(x_train_features, train_categories)
a = svc.predict(x_test_features)
b = pd.Series(a)
b.to_csv("prediction4.csv",header=False, index=False)
svc.score(x_train_features, train_categories)

0.9936247723132969

In [20]:
#0.7667
from sklearn.naive_bayes import MultinomialNB as SVC
svc = SVC(alpha=0.0001)
svc.fit(x_train_features, train_categories)
a = svc.predict(x_test_features)
b = pd.Series(a)
b.to_csv("prediction5.csv",header=False, index=False)
svc.score(x_train_features, train_categories)

0.8586520947176685